In [1]:
import random
import os
import numpy as np
import mxnet as mx

def seed_everything():
    random.seed(42)
    np.random.seed(42)
    mx.random.seed(42)
    
seed_everything()

from deeprenewal._datasets import get_dataset
from gluonts.dataset.util import to_pandas
from deeprenewal._estimator import DeepRenewalEstimator
from deeprenewal._evaluator import IntermittentEvaluator
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.model.r_forecast import RForecastPredictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.evaluation import Evaluator
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions 

import mxnet as mx
import ast
from tqdm import tqdm
from argparse import ArgumentParser
from gluonts.model.forecast import SampleForecast
from croston import croston
from fbprophet import Prophet
from pathlib import Path
from gluonts.model.predictor import Predictor
import pandas as pd

import optuna
import wandb

In [2]:
wandb.init(name="OptunaRunDeepRenewal_v2", project ="DeepRenewal")

wandb: Currently logged in as: manujosephv (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb\run-20200920_164455-93tpu0xr
wandb: Syncing run OptunaRunDeepRenewal_v2


# Functions

In [9]:
lr_epoch_map = {1e-5: 25, 1e-4: 20, 1e-3: 15, 1e-2: 10, 1e-1: 5}
# lr_epoch_map = {1e-5: 3, 1e-4: 3, 1e-3: 3, 1e-2: 3, 1e-1: 3}


def get_trainer_from_trial(trial):
    lr = trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    # epoch_multiplier = 1
    epoch_multiplier = trial.suggest_int("epoch_multiplier", 1, 3)
    epochs = lr_epoch_map[lr] * epoch_multiplier
    bs = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    clip_gradient = trial.suggest_uniform("clip_gradient", 5, 10)
    weight_decay = trial.suggest_categorical("weight_decay", [1e-5, 1e-4, 1e-3, 1e-2])
    trainer = Trainer(ctx=mx.context.gpu(0),
        learning_rate=lr,
        epochs=epochs,
        num_batches_per_epoch=100,
        batch_size=bs,
        clip_gradient=clip_gradient,
        weight_decay=weight_decay,
    )
    return trainer


def get_deeprenewal_from_trial(trial, trainer):
    context_length_multiplier = trial.suggest_categorical(
        "context_length_multiplier", [1, 2, 3, 4]
    )
    context_length = context_length_multiplier * prediction_length
    num_layers = trial.suggest_int("num_layers", 2, 5)
    num_lags = trial.suggest_int("num_lags", 0, 12)
    lags_seq = [] if num_lags==0 else np.arange(1,num_lags+1).tolist()
    num_cells = trial.suggest_categorical("num_cells", [32, 64, 128])
    cell_type = trial.suggest_categorical("cell_type", ["lstm", "gru"])
    dropout_rate = trial.suggest_int("dropout", 1, 5) * 1e-1
    estimator = DeepRenewalEstimator(
        prediction_length=prediction_length,
        context_length=context_length,
        num_layers=num_layers,
        num_cells=num_cells,
        cell_type=cell_type,
        dropout_rate=dropout_rate,
        scaling=True,
        lags_seq=lags_seq,
        freq="1D", ##Freq,
        use_feat_dynamic_real=args.use_feat_dynamic_real,
        use_feat_static_cat=args.use_feat_static_cat,
        use_feat_static_real=args.use_feat_static_real,
        cardinality=cardinality,
        trainer=trainer,
        )
    return estimator


def objective(trial):
    trainer = get_trainer_from_trial(trial)
    estimator = get_deeprenewal_from_trial(trial, trainer)
    try:
        predictor = estimator.train(train_ds, test_ds)
        # ### Generating forecasts
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=test_ds, predictor=predictor, num_samples=100
        )

        tss = list(tqdm(ts_it, total=len(test_ds)))
        forecasts = list(tqdm(forecast_it, total=len(test_ds)))
        # ### Local performance validation
        evaluator = IntermittentEvaluator(quantiles=[0.5])
        agg_metrics, item_metrics = evaluator(
            iter(tss), iter(forecasts), num_series=len(test_ds)
        )
        
        mase = agg_metrics["MASE"]
        cum_mase = agg_metrics["cumMASE"]
        trial.set_user_attr("MASE",mase)
        wandb.log({"MASE": mase,
                  "cumMASE": cum_mase})
    except Exception as e:
        raise e
        trial.set_user_attr("exception", e)
        cum_mase = 1e20
        wandb.log({"MASE": 1e20,
                  "cumMASE": 1e20})
    return cum_mase


# Config

In [10]:
parser = ArgumentParser()
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

# add PROGRAM level args
parser.add_argument('--project-name', type=str, default='deep_renewal_processes')
parser.add_argument('--experiment-tag', type=str, default='deep_renewal_process')
parser.add_argument('--use-cuda', type=bool, default=True)
parser.add_argument('--use-wandb', type=bool, default=True)
parser.add_argument('--log-gradients', type=bool, default=True)
parser.add_argument('--run-optuna-sweep', type=bool, default=True)
parser.add_argument('--datasource', type=str, default="retail_dataset")
parser.add_argument('--model-save-dir', type=str, default="models")

# Trainer specific args
parser.add_argument('--batch_size', type=int, default=256)
parser.add_argument('--learning-rate', type=float, default=1e-5)
parser.add_argument('--max-epochs', type=int, default=25)
parser.add_argument('--number-of-batches-per-epoch', type=int, default=100)
parser.add_argument('--clip-gradient', type=float, default=10)
parser.add_argument('--weight-decay', type=float, default=1e-8)


# Model specific args
parser.add_argument('--context-length-multiplier', type=int, default=2)
parser.add_argument('--num-layers', type=int, default=4)
parser.add_argument('--num-cells', type=int, default=128)
parser.add_argument('--cell-type', type=str, default="gru")
#p% are dropped and set to zero
parser.add_argument('--dropout-rate', type=float, default=0.3)
parser.add_argument('--use-feat-dynamic-real', type=bool, default=True)
parser.add_argument('--use-feat-static-cat', type=bool, default=True)
parser.add_argument('--use-feat-static-real', type=bool, default=False)
parser.add_argument('--scaling', type=bool, default=True)
parser.add_argument('--num-parallel-samples', type=int, default=100)
parser.add_argument('--num-lags', type=int, default=1)
#Only for Deep Renewal Processes
parser.add_argument('--forecast-type', type=str, default="flat")
#Only for Deep AR
parser.add_argument('--distr-output', type=str, default="student_t") #neg_binomial


args = parser.parse_args()
is_gpu = mx.context.num_gpus()>0

# Read in the dataset

In [11]:
dataset = get_dataset(args.datasource, regenerate=False)

In [12]:
prediction_length = dataset.metadata.prediction_length
freq = dataset.metadata.freq
cardinality = ast.literal_eval(dataset.metadata.feat_static_cat[0].cardinality)
train_ds = dataset.train
test_ds = dataset.test

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

df = study.trials_dataframe()
# if len(filters.values())>0:
#     tag = "|".join(filters.values())
# else:
#     tag = 'all'
tag = 'interm'
df.to_csv(f"{tag}_deeprenewal_study_df.csv")

[I 2020-09-20 16:59:13,282] A new study created in memory with name: no-name-ba6a4726-c905-4ae8-bcbd-7029453de8d0


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|████████████████████████████████████████████████| 100/100 [00:40<00:00,  2.48it/s, epoch=1/15, avg_epoch_loss=6.2]
70it [00:20,  3.43it/s, epoch=1/15, validation_avg_epoch_loss=6.09]
100%|███████████████████████████████████████████████| 100/100 [00:43<00:00,  2.31it/s, epoch=2/15, avg_epoch_loss=6.08]
67it [00:18,  3.63it/s, epoch=2/15, validation_avg_epoch_loss=6.04]
100%|███████████████████████████████████████████████| 100/100 [00:45<00:00,  2.22it/s, epoch=3/15, avg_epoch_loss=6.08]
67it [00:17,  3.84it/s, epoch=3/15, validation_avg_epoch_loss=6.11]
100%|███████████████████████████████████████████████| 100/100 [00:43<00:00,  2.30it/s, epoch=4/15, avg_epoch_loss=6.07]
70it [00:17,  4.07it/s, epoch=4/15, validation_avg_epoch_loss=6.04]
100%|███████████████████████████████████████████████| 100/100 [00:45<00:00,  2.18it/s, epoch=5/15, avg_epoch_loss=6.02]
66it [00:17,  3.79it/s, epoch=5/15, validation_avg_epoch_loss=6.04]
100%|███████████████████████████████████████████████| 100/10

MXNetError: Error in operator deeprenewaltrainingnetwork3__mul1: [17:50:08] c:\jenkins\workspace\mxnet-tag\mxnet\src\operator\tensor\../elemwise_op_common.h:135: Check failed: assign(&dattr, vec.at(i)): Incompatible attr in node deeprenewaltrainingnetwork3__mul1 at 1-th input: expected [156], got [1,156]

# Parse Logs

In [6]:
import re
pattern = r'Trial\s(\d{1,2}) finished with value:\s(\d.\d+) and parameters: ({.+})'
trials = []
values=[]
params=[]
with open("temp.txt","r") as f:
    for line in f:
        g = re.search(pattern, line)
        if g:
            trials.append(g.group(1))
            values.append(g.group(2))
            params.append(g.group(3))

In [7]:
import pandas as pd

In [9]:
trial_df = pd.DataFrame({"Trial": trials, "Value":values, "params": params})
trial_df.to_csv("interm_rmse_tuning.csv")